In [1]:
from deep_q_network import DQNAgent
from base_rl_agent_torch import ReplayMemory
from base_neural_model import EstimatorModelBase
import utils
import random
from Data import *

In [2]:
# Fix seeds for reproducibility.
utils.fix_seeds()

In [3]:
env = Data()
env.loadfile("sample.csv")
env.normalize()
env.cluster_K_means(10)

[0.11797752808988764, 0.07865168539325842, 0.15730337078651685, 0.1797752808988764, 0.09550561797752809, 0.11235955056179775, 0.09550561797752809, 0.0449438202247191, 0.06179775280898876, 0.056179775280898875]


In [4]:
MAX_EPISODES = 100
MAX_STEPS = 5
BATCH_SIZE = 32
buffer = ReplayMemory(10000)

# Initiate the agent
model = EstimatorModelBase
agent = DQNAgent(env, model, buffer)

Using cpu


In [5]:
#Train
episode_scores = agent.train(env, MAX_EPISODES, MAX_STEPS, BATCH_SIZE)

(40, [0.4868421052631582, 0.4446640316205534, 0.5561497326203207, 0.48453608247422686, 0.3695652173913043, 0.11034482758620692, None, 0.2075471698113207, None, 0.3515358361774744, None, 0.05494505494505491, None])
6
0.1414213562373095
Episode 0: 0.0414213562373095 	 1
(44, [0.10000000000000019, 0.0, 0.6096256684491979, 0.5360824742268042, None, None, 0.35232067510548515, 0.5471698113207547, None, 0.15358361774744028, None, None, 0.11126961483594865])
8
0.0
Episode 1: -0.1 	 0.9
(143, [None, None, 0.4759358288770053, 0.29896907216494845, None, 0.5586206896551724, 0.540084388185654, 0.15094339622641506, 0.3817034700315458, 0.3899317406143344, 0.3577235772357724, 0.706959706959707, 0.557774607703281])
0
0.20498880527646596
Episode 2: 0.10498880527646595 	 0.81
(13, [0.4131578947368421, None, 0.28877005347593576, 0.4072164948453609, 0.1956521739130435, 0.16206896551724137, 0.2151898734177215, None, 0.2965299684542587, None, None, 0.5494505494505495, 0.20256776034236804])
7
0.24807406984078

TypeError: must be real number, not NoneType

In [ ]:
# Now we evaluate the trained model by taking greedy actions
observation = env.reset() # Current observed state
total_reward = 0
agent.epsilon = 0
while True:
    action = agent.get_action(observation)
    observation, reward, done, info = env.step(state, action)
    total_reward += reward
    if done:
        break;
print("Test Total Reward:", total_reward)

# Saving the network
#agent.to_pickle("somefile.pickle")